## Sagnac Frequency - Backscatter Correction - Compare Setups

Explore the longterm behaviour of the RV sagnac beat

## Imports

In [183]:
import os
import gc
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime, date
from pandas import DataFrame, read_pickle, date_range, concat, read_csv
from obspy import UTCDateTime, read, Trace, Stream, read_inventory
from scipy.signal import hilbert
from scipy.ndimage import gaussian_filter1d

from functions.get_fft import __get_fft
from functions.multitaper_psd import __multitaper_psd
from functions.welch_psd import __welch_psd
from functions.rotate_romy_ZUV_ZNE import __rotate_romy_ZUV_ZNE
from functions.reduce import __reduce
from functions.load_backscatter_data import __load_backscatter_data
from functions.backscatter_correction import __backscatter_correction

from andbro__read_sds import __read_sds
from andbro__readYaml import __readYaml
from andbro__load_FURT_stream import __load_furt_stream

In [184]:
if os.uname().nodename == 'lighthouse':
    root_path = '/home/andbro/'
    data_path = '/home/andbro/kilauea-data/'
    archive_path = '/home/andbro/freenas/'
    bay_path = '/home/andbro/ontap-ffb-bay200/'
    lamont_path = '/home/andbro/lamont/'
elif os.uname().nodename == 'kilauea':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'
    bay_path = '/import/ontap-ffb-bay200/'
    lamont_path = '/lamont/'
elif os.uname().nodename in ['lin-ffb-01', 'ambrym', 'hochfelln']:
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'
    bay_path = '/import/ontap-ffb-bay200/'
    lamont_path = '/lamont/'

## Configurations

In [9]:
config = {}

config['ring'] = "Z"

# config['delta'] = 120

# switch of PDs for monobeams at RZ & new control loop
config['tbeg'] = UTCDateTime("2024-10-23 00:00")
# config['tend'] = UTCDateTime("2024-08-10 00:00")
flim1, flim2 = 553.569, 553.577

# path to Sagnac data
config['path_to_data'] = data_path+"sagnac_frequency/data/compare_methods/backscatter/"

# config['path_to_out_data'] = data_path+"sagnac_frequency/data/"

# path to out figures
config['path_to_figs'] = data_path+"sagnac_frequency/figures/"


In [142]:
class sagnacf:

    import numpy as np
    from obspy import UTCDateTime

    def __init__(self, delta=60):

        self.delta = delta

    def load_data(self, path, filename):
        self.data = read_pickle(path+filename)

    def relative_timing(self, tbeg):
        self.data['time_sec'] = self.data.time1 - UTCDateTime(tbeg) + self.delta/2

    def trim(self, tbeg=None, tend=None):
        if tbeg is not None:
            _df = self.data
            self.data = _df[_df.time1 >= UTCDateTime(tbeg)]
        if tend is not None:
            _df = self.data
            self.data = _df[_df.time2 <= UTCDateTime(tend)]

    def apply_backscatter_correction(self):

        # copy data
        _data = self.data

        # compute unwrapped phase difference
        phase_difference = np.unwrap(_data.f1_phw) - np.unwrap(_data.f2_phw)

        # compute backscatter correction
        _data['fj_bs'], _, _ = self.bs_correction(_data.f1_ac / _data.f1_dc,
                                                  _data.f2_ac / _data.f2_dc,
                                                  phase_difference,
                                                  _data.fj_fs,
                                                  np.nanmedian(_data.fj_fs),
                                                  cm_filter_factor=1.033,
                                                 )
        # reasign updated dataframe
        self.data = _data

    @staticmethod
    def bs_correction(m01, m02, phase0, w_obs, fs0, cm_filter_factor=1.033):

        from numpy import array, sin, cos

        # Correct for bias
        m1 = m01 * ( 1 + m01**2 / 4 )
        m2 = m02 * ( 1 + m02**2 / 4 )

        # angular correction for phase
        phase = phase0 + 0.5 * m1 * m2 * sin( phase0 )

        # compute squares of common-mode modulations
        m2c = ( m1**2 + m2**2 + 2*m1*m2*cos( phase ) ) / 4

        # compute squares of differential-mode modulations
        m2d = ( m1**2 + m2**2 - 2*m1*m2*cos( phase ) ) / 4  ## different angle!

        # correct m2c for gain saturation of a HeNe laser
        # m2c = m2c * ( 1 + ( beta + theta )**2 * fL**2 * I0**2 / ws**2 )
        m2c = m2c * cm_filter_factor

        # compute backscatter correction factor
        M = m2c - m2d + 0.25 * m1**2 * m2**2 * sin(phase)**2

        # correction term
        term = ( 4 + M ) / ( 4 - M )

        # backscatter correction
        correction = -1 * ( term - 1 ) * fs0
        # w_corrected = np.array(w_obs) + correction

        # apply backscatter correction
        w_corrected = array(w_obs) * term

        return w_corrected, correction, term


### Load backscatter data

In [134]:
lbl1 = "Z_T120_hilbert"

bs1 = sagnacf(delta=120)

bs1.load_data(config['path_to_data'], "FJZ_20241023_backscatter_T120_hilbert.pkl")

bs1.relative_timing(config['tbeg'])

bs1.apply_backscatter_correction()

bs1.trim(tbeg=UTCDateTime("2024-10-23 12:00"))

In [135]:
lbl2 = "Z_T60_hilbert"

bs2 = sagnacf(delta=60)

bs2.load_data(config['path_to_data'], "FJZ_20241023_backscatter_T60_hilbert.pkl")

bs2.relative_timing(config['tbeg'])

bs2.apply_backscatter_correction()

bs2.trim(tbeg=UTCDateTime("2024-10-23 12:00"))

In [136]:
lbl3 = "Z_T120_hilbert_ca"

bs3 = sagnacf(delta=120)

bs3.load_data(config['path_to_data'], "FJZ_20241023_backscatter_T120_hilbert_CA.pkl")

bs3.relative_timing(config['tbeg'])

bs3.apply_backscatter_correction()

bs3.trim(tbeg=UTCDateTime("2024-10-23 12:00"))

In [137]:
lbl4 = "Z_T60_hilbert_ca"

bs4 = sagnacf(delta=60)

bs4.load_data(config['path_to_data'], "FJZ_20241023_backscatter_T60_hilbert_CA.pkl")

bs4.relative_timing(config['tbeg'])

bs4.apply_backscatter_correction()

bs4.trim(tbeg=UTCDateTime("2024-10-23 12:00"))

In [138]:
lbl5 = "Z_T60_sine"

bs5 = sagnacf(delta=60)

bs5.load_data(config['path_to_data'], "FJZ_20241023_backscatter_T60_sine.pkl")

bs5.relative_timing(config['tbeg'])

bs5.apply_backscatter_correction()

bs5.trim(tbeg=UTCDateTime("2024-10-23 12:00"))

In [139]:
lbl6 = "Z_T120_sine"

bs6 = sagnacf(delta=120)

bs6.load_data(config['path_to_data'], "FJZ_20241023_backscatter_T120_sine.pkl")

bs6.relative_timing(config['tbeg'])

bs6.apply_backscatter_correction()

bs6.trim(tbeg=UTCDateTime("2024-10-23 12:00"))

In [145]:
lbl7 = "Z_T120_sine_ca"

bs7 = sagnacf(delta=120)

bs7.load_data(config['path_to_data'], "FJZ_20241023_backscatter_T120_sine_CA.pkl")

bs7.relative_timing(config['tbeg'])

bs7.apply_backscatter_correction()

bs7.trim(tbeg=UTCDateTime("2024-10-23 12:00"))

In [163]:
lbl8 = "Z_T60_sine_ca"

bs8 = sagnacf(delta=120)

bs8.load_data(config['path_to_data'], "FJZ_20241023_backscatter_T60_sine_CA.pkl")

bs8.relative_timing(config['tbeg'])

bs8.apply_backscatter_correction()

bs8.trim(tbeg=UTCDateTime("2024-10-23 12:00"))

In [181]:
lbl0 = "Z_T200_hilbert"

bs0 = sagnacf(delta=120)

bs0.load_data(config['path_to_data'], "FJZ_20241023_backscatter_T200_hilbert.pkl")

bs0.relative_timing(config['tbeg'])

bs0.apply_backscatter_correction()

bs0.trim(tbeg=UTCDateTime("2024-10-23 12:00"))

In [169]:
%matplotlib tk

## Compare Hilbert vs. Sine

In [178]:
def __makeplot():

    import matplotlib.pyplot as plt

    Nrow, Ncol = 2, 1

    font = 12

    tscale, tunit = 1/3600, "hour"

    fig, ax = plt.subplots(Nrow, Ncol, figsize=(12, 5), sharex=True)

    plt.subplots_adjust(hspace=0.1)


    ax[0].plot(bs4.data.time_sec*tscale, bs4.data.fj_fs, label=lbl4)
    ax[0].plot(bs2.data.time_sec*tscale, bs2.data.fj_fs, label=lbl2)
    # ax[0].plot(bs1.data.time_sec*tscale, bs1.data.fj_fs, label=lbl1)
    # ax[0].plot(bs3.data.time_sec*tscale, bs3.data.fj_fs, label=lbl3)
    # ax[0].plot(bs6.data.time_sec*tscale, bs6.data.fj_fs, label=lbl6)
    ax[0].plot(bs5.data.time_sec*tscale, bs5.data.fj_fs, label=lbl5)
    # ax[0].plot(bs7.data.time_sec*tscale, bs7.data.fj_fs, label=lbl7)
    ax[0].plot(bs8.data.time_sec*tscale, bs8.data.fj_fs, label=lbl8)


    ax[1].plot(bs4.data.time_sec*tscale, bs4.data.fj_bs, label=lbl4)
    ax[1].plot(bs2.data.time_sec*tscale, bs2.data.fj_bs, label=lbl2)
    # ax[1].plot(bs1.data.time_sec*tscale, bs1.data.fj_bs, label=lbl1)
    # ax[1].plot(bs3.data.time_sec*tscale, bs3.data.fj_bs, label=lbl3)
    # ax[1].plot(bs6.data.time_sec*tscale, bs6.data.fj_bs, label=lbl6)
    ax[1].plot(bs5.data.time_sec*tscale, bs5.data.fj_bs, label=lbl5)
    ax[1].plot(bs8.data.time_sec*tscale, bs8.data.fj_bs, label=lbl8)

    ax[0].legend(ncol=2)

    ax[0].set_ylabel("$\delta$f (Hz) w\ bs ", fontsize=font)
    ax[1].set_ylabel("$\delta$f(Hz) w\o bs ", fontsize=font)

    ax[1].set_xlabel(f"Time ({tunit})", fontsize=font)

    for j in range(Nrow):

        ax[j].grid(which="both", ls="--", color="grey", alpha=0.5, zorder=0)
        ax[j].ticklabel_format(useOffset=False)

    ax[0].set_ylim(553.496, 553.499)
    ax[1].set_ylim(553.496, 553.499)

    for _k, ll in enumerate(['(a)', '(b)']):
        ax[_k].text(0.005, 0.97, ll, ha="left", va="top", transform=ax[_k].transAxes, fontsize=font+1)

    plt.show();
    return fig

fig = __makeplot();

## Compare T60 vs. T120

In [182]:
def __makeplot():

    import matplotlib.pyplot as plt

    Nrow, Ncol = 2, 1

    font = 12

    tscale, tunit = 1/3600, "hour"

    fig, ax = plt.subplots(Nrow, Ncol, figsize=(12, 5), sharex=True)

    plt.subplots_adjust(hspace=0.1)

    ax[0].plot(bs2.data.time_sec*tscale, bs2.data.fj_fs, label=lbl2)
    ax[0].plot(bs1.data.time_sec*tscale, bs1.data.fj_fs, label=lbl1)

    ax[0].plot(bs5.data.time_sec*tscale, bs5.data.fj_fs, label=lbl5)
    ax[0].plot(bs6.data.time_sec*tscale, bs6.data.fj_fs, label=lbl6)

    # ax[0].plot(bs0.data.time_sec*tscale, bs0.data.fj_fs, label=lbl0)

    ax[1].plot(bs2.data.time_sec*tscale, bs2.data.fj_bs, label=lbl2)
    ax[1].plot(bs1.data.time_sec*tscale, bs1.data.fj_bs, label=lbl1)

    ax[1].plot(bs5.data.time_sec*tscale, bs5.data.fj_bs, label=lbl5)
    ax[1].plot(bs6.data.time_sec*tscale, bs6.data.fj_bs, label=lbl6)

    # ax[1].plot(bs0.data.time_sec*tscale, bs0.data.fj_bs, label=lbl0)

    ax[0].legend(ncol=2)

    ax[0].set_ylabel("$\delta$f (Hz) w\ bs ", fontsize=font)
    ax[1].set_ylabel("$\delta$f(Hz) w\o bs ", fontsize=font)

    ax[1].set_xlabel(f"Time ({tunit})", fontsize=font)

    for j in range(Nrow):

        ax[j].grid(which="both", ls="--", color="grey", alpha=0.5, zorder=0)
        ax[j].ticklabel_format(useOffset=False)

    ax[0].set_ylim(553.496, 553.499)
    ax[1].set_ylim(553.496, 553.499)

    for _k, ll in enumerate(['(a)', '(b)']):
        ax[_k].text(0.005, 0.97, ll, ha="left", va="top", transform=ax[_k].transAxes, fontsize=font+1)

    plt.show();
    return fig

fig = __makeplot();

## Compare Frequency Band

In [194]:
lbl10 = "Z_T60_hilbert_2Hz"

bs10 = sagnacf(delta=60)

bs10.load_data(config['path_to_data'], "FJZ_20241023_backscatter_T60_hilbert.pkl")

bs10.relative_timing(config['tbeg'])

bs10.apply_backscatter_correction()

bs10.trim(tbeg=UTCDateTime("2024-10-23 12:00"))

In [195]:
lbl11 = "Z_T200_hilbert_5Hz"

bs11 = sagnacf(delta=60)

bs11.load_data(config['path_to_data'], "FJZ_20241023_backscatter_T60_hilbert5Hz.pkl")

bs11.relative_timing(config['tbeg'])

bs11.apply_backscatter_correction()

bs11.trim(tbeg=UTCDateTime("2024-10-23 12:00"))

In [196]:
lbl12 = "Z_T200_hilbert_1Hz"

bs12 = sagnacf(delta=60)

bs12.load_data(config['path_to_data'], "FJZ_20241023_backscatter_T60_hilbert1Hz.pkl")

bs12.relative_timing(config['tbeg'])

bs12.apply_backscatter_correction()

bs12.trim(tbeg=UTCDateTime("2024-10-23 12:00"))

In [198]:
def __makeplot():

    import matplotlib.pyplot as plt

    Nrow, Ncol = 2, 1

    font = 12

    tscale, tunit = 1/3600, "hour"

    fig, ax = plt.subplots(Nrow, Ncol, figsize=(12, 5), sharex=True)

    plt.subplots_adjust(hspace=0.1)

    ax[0].plot(bs11.data.time_sec*tscale, bs11.data.fj_fs, label=lbl11)
    ax[0].plot(bs10.data.time_sec*tscale, bs10.data.fj_fs, label=lbl10)
    ax[0].plot(bs12.data.time_sec*tscale, bs12.data.fj_fs, label=lbl12)

    ax[1].plot(bs11.data.time_sec*tscale, bs11.data.fj_bs, label=lbl11)
    ax[1].plot(bs10.data.time_sec*tscale, bs10.data.fj_bs, label=lbl10)
    ax[1].plot(bs12.data.time_sec*tscale, bs12.data.fj_bs, label=lbl12)


    ax[0].legend(ncol=2)

    ax[0].set_ylabel("$\delta$f (Hz) w\ bs ", fontsize=font)
    ax[1].set_ylabel("$\delta$f(Hz) w\o bs ", fontsize=font)

    ax[1].set_xlabel(f"Time ({tunit})", fontsize=font)

    for j in range(Nrow):

        ax[j].grid(which="both", ls="--", color="grey", alpha=0.5, zorder=0)
        ax[j].ticklabel_format(useOffset=False)

    ax[0].set_ylim(553.496, 553.499)
    ax[1].set_ylim(553.496, 553.499)

    for _k, ll in enumerate(['(a)', '(b)']):
        ax[_k].text(0.005, 0.97, ll, ha="left", va="top", transform=ax[_k].transAxes, fontsize=font+1)

    plt.show();
    return fig

fig = __makeplot();

In [200]:
lblx = "Z_T200_hilbert_1Hz"

bsx = sagnacf(delta=60)

bsx.load_data("/import/kilauea-data/sagnac_frequency/data/backscatter/", "FJZ_20241023_backscatter_T10_hilbert.pkl")

bsx.relative_timing(config['tbeg'])

bsx.apply_backscatter_correction()

bsx.trim(tbeg=UTCDateTime("2024-10-23 12:00"))

In [207]:
plt.plot(bsx.data.time_sec, bsx.data.fj_fs)

,time1,time2,fj_fs,fj_ac,fj_dc,fj_ph,fj_st,fj_phw,f1_fs,f1_ac,...,f2_ac,f2_dc,f2_ph,f2_st,f2_phw,w_s,bscorrection,term,time_sec,fj_bs
0,2024-10-23T12:00:00.000000Z,2024-10-23T12:00:10.000000Z,553.497588,0.343729,0.045346,-2.807973,0.003929,-2.807973,553.490072,0.000168,...,0.000217,0.265162,-0.092862,0.270158,-0.092862,553.497577,0.000012,1.0,43230.0,553.497577
1,2024-10-23T12:00:10.000000Z,2024-10-23T12:00:20.000000Z,553.497499,0.343451,0.045348,-2.966943,0.003897,-2.966943,553.490158,0.000170,...,0.000217,0.265152,-0.259208,0.384112,-0.259208,553.497488,0.000011,1.0,43240.0,553.497488
2,2024-10-23T12:00:20.000000Z,2024-10-23T12:00:30.000000Z,553.497403,0.343119,0.045350,-3.126171,0.003847,-3.126171,553.538421,0.000170,...,0.000217,0.265146,-0.418836,0.345572,-0.418836,553.497392,0.000011,1.0,43250.0,553.497392
3,2024-10-23T12:00:30.000000Z,2024-10-23T12:00:40.000000Z,553.497041,0.342685,0.045350,-3.285753,0.004331,2.997433,553.500115,0.000170,...,0.000217,0.265131,-0.581422,0.409787,-0.581422,553.497031,0.000010,1.0,43260.0,553.497031
4,2024-10-23T12:00:40.000000Z,2024-10-23T12:00:50.000000Z,553.497838,0.342344,0.045349,-3.445406,0.003375,2.837780,553.475043,0.000170,...,0.000217,0.265122,-0.744980,0.372708,-0.744980,553.497828,0.000010,1.0,43270.0,553.497828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,2024-10-23T23:59:10.000000Z,2024-10-23T23:59:20.000000Z,553.497052,0.304429,0.044608,-62.110485,0.002210,0.721368,553.500858,0.000570,...,0.000128,0.265112,-60.216060,0.761013,2.615794,553.496937,0.000115,1.0,86380.0,553.496937
356,2024-10-23T23:59:20.000000Z,2024-10-23T23:59:30.000000Z,553.496978,0.304666,0.044612,-62.289501,0.002642,0.542352,553.494035,0.000570,...,0.000127,0.265171,-60.385614,0.656992,2.446240,553.496865,0.000113,1.0,86390.0,553.496865
357,2024-10-23T23:59:30.000000Z,2024-10-23T23:59:40.000000Z,553.497431,0.304768,0.044611,-62.468039,0.002411,0.363814,553.504084,0.000570,...,0.000127,0.265173,-60.541916,0.551941,2.289937,553.497324,0.000108,1.0,86400.0,553.497324
358,2024-10-23T23:59:40.000000Z,2024-10-23T23:59:50.000000Z,553.496897,0.304780,0.044611,-62.646647,0.001699,0.185206,553.491068,0.000572,...,0.000126,0.265168,-60.727081,5.648536,2.104772,553.496789,0.000108,1.0,86410.0,553.496789
